In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import genfromtxt
from numpy import linalg as LA
import scipy as sp
import sympy

In [70]:
input_data = pd.read_csv('a6n.csv', index_col=0)
G = nx.Graph(input_data.values)

### how many driver node we have?

In [11]:
def number_of_driver_nodes(G):

    N = G.number_of_nodes()
    A = (nx.adjacency_matrix(G)).todense()  # get adjacency matrix A of G
    all_eigs = LA.eigvals(A)           # get eigenvalues of A
   
    lambda_i = list(set(np.round(all_eigs,8)))
    #lambda_i = list(set(all_eigs))

    driver_nodes_num = -1
    lambda_m = -1
    IN = np.eye(N)
    n = len(lambda_i)
    miu_lambda =np.zeros(n)

    for i in range(0,n):
    
        miu_lambda[i] = N - LA.matrix_rank(lambda_i[i] * IN - A, tol=1E-6)
        if miu_lambda[i] > driver_nodes_num:
            driver_nodes_num = miu_lambda[i]
            lambda_m = lambda_i[i]


    return (driver_nodes_num, lambda_m)


### Which node is a driver node?

In [103]:
def get_driver_nodes(G):
    
    N = G.number_of_nodes()
    A = (nx.adjacency_matrix(G)).todense()  # get adjacency matrix A of G
    all_eigs = LA.eigvals(A)           # get eigenvalues of A
    
    lambda_i = list(set(np.round(all_eigs,8)))
    #lambda_i = list(set(all_eigs))

    driver_nodes_num = -1
    lambda_m = -1
    IN = np.eye(N)
    n = len(lambda_i)
    miu_lambda =np.zeros(n)

    for i in range(0,n):
    
        miu_lambda[i] = N - LA.matrix_rank(lambda_i[i] * IN - A, tol=1E-8)
        if miu_lambda[i] > driver_nodes_num:
            driver_nodes_num = miu_lambda[i]
            lambda_m = lambda_i[i]

        
    middle_matrix = lambda_m * np.eye(N) - A   # get the middle matrix A - \lambda * I_N
    middle_matrix = np.round(middle_matrix, 8)
    reduced_matrix,pivot_array1=sympy.Matrix(middle_matrix).rref()
    
    reduced_matrix_array = np.array(reduced_matrix).astype(np.float64)
    reduced_matrix_array_transpose=np.matrix.transpose(reduced_matrix_array)
    _, pivot_array2 = sympy.Matrix(reduced_matrix_array_transpose).T.rref()
    all_nodes = G.nodes() 
    pivot_nodes = [all_nodes[i] for i in pivot_array2]
    t=0
    N_d = N-len(pivot_array2)
    driver_nodes = np.zeros(N_d)
    for i in all_nodes:
        if i in pivot_array2:
            pass
        else:
            driver_nodes[t] = i
            t = t+1
           
    return (driver_nodes_num, driver_nodes)


## Example

In [104]:
mat = np.array(
    [[-1.0, 1.0, 1.0,1.0,1.0,1.0],
      [1.0, 0.0, 0.0,0.0,0.0, 0.0],
      [1.0, 0.0, 0.0,0.0,0.0 ,0.0], 
      [1.0, 0.0, 0.0,-1.0,0.0,0.0],
      [1.0, 0.0, 0.0,0.0,-1.0,1.0],
      [1.0, 0.0, 0.0,0.0,1.0,-1.0]
    ])
G2 = nx.Graph(mat)

In [105]:
input_data = pd.read_csv('Book2.csv', index_col=0)
G3 = nx.Graph(input_data.values)

In [108]:
number_of_driver_nodes(G3)

(2.0, 0.0)

In [109]:
get_driver_nodes(G3)

(2.0, array([2., 5.]))